<a href="https://colab.research.google.com/github/TCFSH69/EEG-Machine-Learning/blob/main/deap_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1	Fp1
2	AF3
3	F3
4	F7
5	FC5
6	FC1
7	C3
8	T7
9	CP5
10	CP1
11	P3
12	P7
13	PO3
14	O1
15	Oz
16	Pz
17	Fp2
18	AF4
19	Fz
20	F4
21	F8
22	FC6
23	FC2
24	Cz
25	C4
26	T8
27	CP6
28	CP2
29	P4
30	P8
31	PO4
32	O2

In [ ]:
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
# Constants
CHANNEL_NUM = 32
USER_NUM = 32
TRAINING_NUM = 24
VAL_NUM = USER_NUM - TRAINING_NUM
SAMPLE_POINT = 8064

In [ ]:
'''
Load data from 32 participants and resize data & label vectors
'''
def load_data():
    eeg_data   = []
    eeg_labels = []
    for file_id in range(1, USER_NUM + 1):
        with open(f'deap-dataset\data_preprocessed_python\s{file_id:02d}.dat', 'rb') as file:
            data = pickle.load(file)
            eeg_data.append(data['data'][:, :CHANNEL_NUM, :])
            eeg_labels.append(data['labels'][:, :1]) # only valence

    eeg_data   = np.array(eeg_data)
    eeg_labels = np.array(eeg_labels)

    eeg_data   = eeg_data.reshape(-1, CHANNEL_NUM, SAMPLE_POINT) # (32 * 40, 32, 8064)
    eeg_labels = eeg_labels.reshape(-1, 1)      # (32 * 40, 1)

    return eeg_data, eeg_labels




In [ ]:
eeg_data, eeg_labels = load_data()

# Standardize each channel
scaler = StandardScaler()
eeg_data = eeg_data.reshape(-1, SAMPLE_POINT)
eeg_data = scaler.fit_transform(eeg_data)
eeg_data = eeg_data.reshape(-1, CHANNEL_NUM, SAMPLE_POINT)


In [ ]:
# 24 participants for training dataset
# 8 participants for verification dataset
X_train, X_val = eeg_data[:TRAINING_NUM * CHANNEL_NUM], eeg_data[TRAINING_NUM * CHANNEL_NUM:]
y_train, y_val = eeg_labels[:TRAINING_NUM * CHANNEL_NUM], eeg_labels[TRAINING_NUM * CHANNEL_NUM:]

In [ ]:
# Model training
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(CHANNEL_NUM, SAMPLE_POINT)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae'])

history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_val, y_val))

In [ ]:
# Model prediction
correct = 0
for i in range(VAL_NUM * CHANNEL_NUM):
    predict = model.predict(X_val[i].reshape(1, CHANNEL_NUM, SAMPLE_POINT))[0][0]
    real = y_val[i][0]
    correct += ((predict >= 5 and real >= 5) or (predict < 5 and real < 5))

print(f"Accuracy: {correct / (VAL_NUM * CHANNEL_NUM) * 100}%")

Accuracy: 80.859375%
